In [17]:
import jax
import jax.numpy as jnp
import inference.mlp as mlp
import subprocess

In [12]:
c = 1
h = 30
w = 40
sizes = [512, 256]
output = 3

cfg = mlp.MLP_config(
    c = c, 
    h = h,
    w = w,
    sizes = sizes,
    output = output,
    name = "test_mult_outputs"
)

mlp.save_cfg(cfg, "weights/" + cfg.name)

key = jax.random.PRNGKey(0)

weights = mlp.qs_mlp(cfg.c, cfg.h, cfg.w, cfg.sizes, key, cfg.output)

In [13]:
import pickle

# Specify the file path
file_path = "weights/test_mult_outputs.pkl"

# Save the weights to the file
with open(file_path, "wb") as file:
    pickle.dump(weights, file)

# Load the weights from the file
with open(file_path, "rb") as file:
    loaded_weights = pickle.load(file)

# Check if the loaded weights are the same as the original weights
for i in range(len(weights)):
    for j in range(len(weights[i])):
        assert jnp.allclose(weights[i][j], loaded_weights[i][j])

In [14]:
import os
file_path = "weights/test_mult_outputs.bin"
mlp.mlp_serialize_binary(weights, file_path)
print(os.path.getsize(file_path))


5445672


In [15]:
with open('weights/test_mult_outputs.pkl', 'rb') as file:
    loaded_weights = pickle.load(file)
    
x = jnp.ones((2, c, h, w), dtype=jnp.float32).flatten()
output = mlp.mlp_forward(loaded_weights, x)


In [6]:
output

Array([7.180218e-05, 3.698504e-03, 2.177698e-02], dtype=float32)

In [16]:
x = jnp.zeros((2, c, h, w), dtype=jnp.float32).flatten()
output = mlp.mlp_forward(loaded_weights, x)
output

Array([0.        , 0.        , 0.00130664], dtype=float32)

In [7]:
import struct

float_value = output[0]

byte_representation = struct.pack('f', float_value)
integer_representation = struct.unpack('I', byte_representation)[0]
integer_representation


949392512

In [8]:
#now convert the integer back to float
integer_representation = 1008862647
byte_representation = struct.pack('I', integer_representation)
float_value = struct.unpack('f', byte_representation)[0]
float_value

0.009889057837426662